# API do Spotify
Nesse notebook, usei a API do Spotify para buscar as músicas de determinado artista. A API do Spotify funciona com OAuth2.0, um protocolo de autenticação que requer um token de acesso para realizar as requisições. Os passos que realizem nesse notebook foram os seguintes:
 - Requisitar o token de acesso
 - Requisitar os albums do artista
 - Requisitar as músicas de cada álbum
 - Trazer esses dados para um dataframe do pandas

### Primeiro passo, conseguir o token de acesso
Para ter acesso ao token, é necessário fazer uma requisição POST para o URL específico para isso. Essa requisição deverá ter um body e um header que contém informações sobre a aplicação, como ID e Secret. Mais informações [aqui](https://developer.spotify.com/documentation/general/guides/authorization/client-credentials/).

In [1]:
# Importar algumas bibliotecas
import pandas as pd
import requests                            # para realizar as requisições
import base64                              # para codificar em base64
import json                                # para ler os resultados
from config import clientID, clientSecret  # ID e Secret armazenados em outro arquivo

In [2]:
# Passar o ID e Secret para base64 para ir no Header da requisição
message = f"{clientID}:{clientSecret}"
message_bytes = message.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

In [3]:
authUrl = "https://accounts.spotify.com/api/token"        # endpoint da API
authHeader = {'Authorization':'Basic '+ base64_message}   # header da requisição
authData = {'grant_type':'client_credentials'}            # body

In [4]:
res = requests.post(authUrl, headers=authHeader, data=authData)    # Requisição POST, com header e body
responseObject = res.json()                                        # Resultado da requisição
access_token = responseObject['access_token']                      # O token de acesso

### Segundo passo, requisitar os albuns do artista
Com o token, agora é possível buscar dados da biblioteca do Spotify. A primeira requisição irá buscar os albums do artista, juntamente com seu id que será utilizado no futuro. Essas informações ficarão armazenadas no dataframe `albums`

In [5]:
artistId = "3h7RaVXBvdSNa7LXQtVYqH"                                                      # ID do artista
albunsUrl = f"https://api.spotify.com/v1/artists/{artistId}/albums?include_groups=album" # endpoint da API
header = {'Authorization': 'Bearer '+ access_token}                                      # header da requisição

In [6]:
res = requests.get(albunsUrl, headers=header)
albumsObject = res.json()

In [7]:
albumsObject

{'href': 'https://api.spotify.com/v1/artists/3h7RaVXBvdSNa7LXQtVYqH/albums?offset=0&limit=20&include_groups=album',
 'items': [{'album_group': 'album',
   'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3h7RaVXBvdSNa7LXQtVYqH'},
     'href': 'https://api.spotify.com/v1/artists/3h7RaVXBvdSNa7LXQtVYqH',
     'id': '3h7RaVXBvdSNa7LXQtVYqH',
     'name': 'Armandinho',
     'type': 'artist',
     'uri': 'spotify:artist:3h7RaVXBvdSNa7LXQtVYqH'}],
   'available_markets': ['AD',
    'AE',
    'AG',
    'AL',
    'AM',
    'AO',
    'AR',
    'AT',
    'AU',
    'AZ',
    'BA',
    'BB',
    'BD',
    'BE',
    'BF',
    'BG',
    'BH',
    'BI',
    'BJ',
    'BN',
    'BO',
    'BR',
    'BS',
    'BT',
    'BW',
    'BY',
    'BZ',
    'CA',
    'CD',
    'CG',
    'CH',
    'CI',
    'CL',
    'CM',
    'CO',
    'CR',
    'CV',
    'CW',
    'CY',
    'CZ',
    'DE',
    'DJ',
    'DK',
    'DM',
    'DO',
    'DZ',
    'EC',
    'EE',
 

In [8]:
# Salvar uma lista com o nome dos albums e outra com o ID, para alimentar o dataframe
albumNames = [album['name'] for album in albumsObject['items']]
albumIds = [album['id'] for album in albumsObject['items']]

In [9]:
albuns = pd.DataFrame({
    'albumNames' : albumNames,
    'albumIds' : albumIds
})

In [10]:
albuns.head(3)  

,albumNames,albumIds
0,Acústico (Ao Vivo),6lmVZjhGk3WXhilmHHNbdR
1,Sol Loiro,0tyhq2pv2sTd4JeIZqeOwR
2,Ao Vivo em Buenos Aires,6awNZk5um5gkMuPZ8Kg8BD


Agora vou remover da base os albums ao vivo, por repetirem as músicas dos outros albums.

In [11]:
albuns = albuns[~albuns.albumNames.str.contains("Ao Vivo")]                       # remove os albums que contem "Ao Vivo"
albuns = albuns[albuns["albumNames"] != 'Armandinho Naturalmente'].reset_index()  # remove esse álbum, que também é ao vivo

In [12]:
albuns

,index,albumNames,albumIds
0,1,Sol Loiro,0tyhq2pv2sTd4JeIZqeOwR
1,3,Volume 5,6E593UtMC9NKFZisq6VO3k
2,5,Armandinho,1Nearg41kDiDoe4jHQpOAT
3,6,Casinha,4LEeNDHmnXsM04yoJARCYu
4,7,Semente,5ankef7uixssXhd76nE2Aw


### Terceiro passo, pegar as músicas a partir dos albuns
Com os albums e suas IDs, podemos requisitar as músicas que constam em cada um deles. A estrutura dessa requisição é semelhante a anterior, só muda o endpoint. Os nomes das músicas e seus albums ficarão armazenadas do dataframe `df`

In [13]:
# Defini uma função que, a partir da ID do album, retorna uma lista com as faixas
def getAlbumTracks(albumId):
    tracksUrl = f"https://api.spotify.com/v1/albums/{albumId}/tracks"   # endpoint da api
    header = {'Authorization': 'Bearer '+ access_token}                 # header da requisição, com o mesmo token de antes

    res = requests.get(tracksUrl, headers=header)                       # requisição
    tracksObject = res.json()                                           # resultado da requisição
    
    trackNames = [track['name'] for track in tracksObject['items']]     # nomes das faixas armazenados nessa lista
    
    return trackNames

In [14]:
# Fiz um loop que vai pegar o dataframe albums e vai iterar ele, pegando as músicas de cada um e armazendando no dataframe df

songsList = []
albumsList = []

for index, row in albuns.iterrows():
    albumSongs = getAlbumTracks(albuns['albumIds'][index])
    for i, r in enumerate(albumSongs):
        songsList.append(albumSongs[i])
        albumsList.append(row['albumNames'])
        
df = pd.DataFrame({
    'album': albumsList,
    'songs': songsList
})

In [15]:
df.sample(10)

,album,songs
40,Casinha,Paulinha
4,Sol Loiro,Nada Mudou
47,Casinha,O Leãozinho
12,Sol Loiro,O Que Meu Pai Falou Pra Mim
59,Semente,I Can't Say
52,Semente,Semente
25,Volume 5,As Festas Que Eu Vou
42,Casinha,Analua
20,Volume 5,Mulher do Brother
38,Casinha,Pescador


Como resultado, temos um dataframe com as músicas do artista e seus respectivos albums, retirados da biblioteca do Spotify a partir da API